# 3. 피처 엔지니어링 및 데이터 통합

## 1) 공구별 아파트 단지 분류 및 지도 시각화 (KMeans 클러스터링)

In [ ]:
# 공구별 아파트 단지 분류 (KMeans 클러스터링)
apt_locations = apt_df.groupby('aptNm')[['latitude', 'longitude']].mean().reset_index()
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
apt_locations['cluster'] = kmeans.fit_predict(apt_locations[['latitude', 'longitude']])
apt_locations['zone'] = apt_locations['cluster'].apply(lambda x: f"zone {x+1}")

# folium 지도 시각화 (선택사항)
map_center = [apt_locations['latitude'].mean(), apt_locations['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=12)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen']
for idx, row in apt_locations.iterrows():
    cluster_num = int(row['zone'].split()[1])
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=colors[cluster_num-1],
        fill=True,
        fill_color=colors[cluster_num-1],
        fill_opacity=0.8,
        popup=f"{row['aptNm']} - {row['zone']}"
    ).add_to(m)
m.save("10gonggu_map.html")

# apt_df에 zone 정보 병합
apt_df = apt_df.merge(apt_locations[['aptNm', 'zone']], on='aptNm', how='left')


## 2) 거리 기반 Feature 생성

In [ ]:
# # 아파트와 교통/편의시설 간의 '최단 거리' (BallTree 이용)
# station_tree = BallTree(np.deg2rad(transport_df[['trans_latitude', 'trans_longitude']]), metric='haversine')
# facility_tree = BallTree(np.deg2rad(convenience_df[['conv_latitude', 'conv_longitude']]), metric='haversine')
# apt_coords = np.deg2rad(apt_df[['latitude', 'longitude']])
# apt_df['dist_to_nearest_station_m'] = station_tree.query(apt_coords, k=1)[0].flatten() * 6371000
# apt_df['dist_to_nearest_facility_m'] = facility_tree.query(apt_coords, k=1)[0].flatten() * 6371000

# # 일정 반경(예: 300m) 내 편의시설/교통시설 개수 (KDTree 이용)
# radius_km = 0.3
# earth_radius_km = 6371.0

# def latlon_to_radians(df, lat_col, lon_col):
#     return np.radians(df[[lat_col, lon_col]].values)

# def count_nearby_by_kdtree(apt_coords, facility_coords, radius_km):
#     radius_rad = radius_km / earth_radius_km
#     tree = cKDTree(facility_coords)
#     counts = tree.query_ball_point(apt_coords, radius_rad)
#     return [len(c) for c in counts]

# apt_coords_rad = latlon_to_radians(apt_df, 'latitude', 'longitude')
# conv_coords_rad = latlon_to_radians(convenience_df, 'conv_latitude', 'conv_longitude')
# trans_coords_rad = latlon_to_radians(transport_df, 'trans_latitude', 'trans_longitude')

# apt_df['nearby_conveniences'] = count_nearby_by_kdtree(apt_coords_rad, conv_coords_rad, radius_km)
# apt_df['nearby_transports'] = count_nearby_by_kdtree(apt_coords_rad, trans_coords_rad, radius_km)


## 3) 카테고리 Feature 인코딩

In [ ]:
# # 브랜드 추출 (예: 더샵, 푸르지오, 힐스테이트, 자이)
# apt_df['brand'] = apt_df['aptNm'].str.extract(r'(더샵|푸르지오|힐스테이트|자이)')
# # zone, brand 변수 원-핫 인코딩 (첫번째 범주는 포함)
# apt_df = pd.get_dummies(apt_df, columns=['zone','brand'], drop_first=False)

## 4) GTX-B 이슈 더미 변수 생성

In [ ]:
# # GTX 이슈 데이터: 연도와 월을 합쳐서 year_month 생성
# gtx_df['year_month'] = pd.to_datetime(gtx_df['연도'].astype(str) + '-' + 
#                                         gtx_df['월'].astype(str).str.zfill(2))
# gtx_events = gtx_df['year_month'].unique()
# # apt_df의 year_month은 Period 타입이므로 Timestamp로 변환하여 비교
# apt_df['year_month_ts'] = apt_df['dealDate'].dt.to_period('M').dt.to_timestamp()
# apt_df['gtx_issue'] = apt_df['year_month_ts'].isin(gtx_events).astype(int)
# apt_df.drop(columns='year_month_ts', inplace=True)

## 5) 금리 데이터 통합

In [ ]:
# # 1. dealDate를 datetime 타입으로 변환한 후,
# apt_df['dealDate'] = pd.to_datetime(apt_df['dealDate'])

# # 2. dealDate에서 "YYYY-MM" 형식의 문자열을 생성 (단 한 번만 실행)
# apt_df['year_month'] = apt_df['dealDate'].dt.strftime('%Y-%m')

# # 확인
# print(apt_df['year_month'].head())
# print(apt_df['year_month'].dtype)

In [ ]:
# # 1. apt_df의 날짜 및 연월 컬럼 재생성 (Period 타입)
# apt_df['dealDate'] = pd.to_datetime(apt_df['dealDate'])
# apt_df['year_month'] = apt_df['dealDate'].dt.to_period('M')  # Period 타입 생성

# # 2. rate_df의 날짜 컬럼 처리: "year_month"를 Period 타입으로 변환 및 이전 달 계산
# rate_df['year_month'] = pd.to_datetime(rate_df['year_month'], format='%Y-%m').dt.to_period('M')
# rate_df['prev_year_month'] = rate_df['year_month'] - 1

# # 3. 이전 달 기준 금리 산출 (정책금리와 시장금리 각각 그룹화하여 마지막 값을 취함)
# policy_rate_monthly = rate_df.groupby('prev_year_month')['Policy Rate'].last().reset_index()
# policy_rate_monthly.rename(columns={'prev_year_month': 'year_month'}, inplace=True)

# market_rate_monthly = rate_df.groupby('prev_year_month')['Market Interest Rate'].last().reset_index()
# market_rate_monthly.rename(columns={'prev_year_month': 'year_month'}, inplace=True)

# # 4. apt_df와 금리 데이터 병합 (양쪽 모두 "year_month"가 Period 타입)
# apt_df = apt_df.merge(policy_rate_monthly, on='year_month', how='left')
# apt_df = apt_df.merge(market_rate_monthly, on='year_month', how='left')

# # 5. 결측치 보정 (ffill, bfill)
# apt_df['Policy Rate'] = apt_df['Policy Rate'].fillna(method='ffill').fillna(method='bfill')
# apt_df['Market Interest Rate'] = apt_df['Market Interest Rate'].fillna(method='ffill').fillna(method='bfill')


## 6) 주택 보급률 데이터 통합

In [ ]:
# # (1) 인구 데이터 정제
# raw_pop_df = pd.read_excel(population_excel, sheet_name="데이터")
# clean_pop_df = raw_pop_df.iloc[1:].reset_index(drop=True)
# clean_pop_df.columns = ['Region', 'Category'] + [str(year) for year in range(2015, 2025)]
# clean_pop_df['Region'] = clean_pop_df['Region'].str.strip()
# population_df = clean_pop_df[clean_pop_df['Category'] == '총인구수 (명)'].drop(columns='Category')
# for col in population_df.columns[1:]:
#     population_df[col] = pd.to_numeric(population_df[col], errors='coerce')

# # (2) 주택 수 데이터 (IFEZ 제공)
# housing_data = pd.DataFrame({
#     "year": [str(y) for y in range(2015, 2025)],
#     "HousingCount": [45856, 50375, 55705, 57525, 59308, 63862, 67828, 67828, 71278, 71719]
# })

# # (3) 주택 보급률 계산
# population_totals = population_df.set_index('Region').loc[:, '2015':'2024'].sum()
# estimated_households = population_totals / 2.3
# housing_data['TotalPopulation'] = housing_data['year'].map(population_totals.to_dict())
# housing_data['EstimatedHouseholds'] = housing_data['year'].map(estimated_households.to_dict())
# housing_data['HousingSupplyRate(%)'] = (housing_data['HousingCount'] / housing_data['EstimatedHouseholds']) * 100

# # (4) apt_df와 주택 보급률 병합 (apt_df의 연도 변수 사용)
# # apt_df의 'year' 컬럼은 int32 타입이므로 문자열로 변환
# apt_df['year'] = apt_df['year'].astype(str)
# apt_df = apt_df.merge(housing_data[['year', 'HousingSupplyRate(%)']], on='year', how='left')
# apt_df.drop(columns='year', inplace=True)

## 7) 최종 데이터 저장

In [ ]:
# apt_df.sort_values('dealDate', inplace=True)
# apt_df.to_csv('final_data_merged.csv', index=False, encoding='utf-8')
# apt_df.to_csv('final_data_merged_sig.csv', index=False, encoding='utf-8-sig')
# print("최종 통합 데이터 저장 완료!")

## 8) 최종 데이터 확인

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# df = pd.read_csv('final_data_merged.csv')

## 9) 변수별 역할 및 중요성

- **거래 관련 변수**:  
  - `dealAmount`, `excluUseAr`, `pricePerpy`, `buildYear`, `floor` 등은 아파트 가격 결정에 직접적인 영향을 주는 핵심 변수로, 로그 변환 및 이상치 제거 등을 통해 정규화된 분포로 모델링에 활용됩니다.
  
- **위치 및 접근성 변수**:  
  - `latitude`, `longitude`, `trans_latitude`, `conv_latitude` 등은 부동산의 지리적 위치와 주변 인프라(교통, 편의시설)와의 거리 측정을 통해, 입지 조건과 접근성을 반영합니다.
  
- **경제 및 거시 변수**:  
  - `Policy Rate`와 `Market Interest Rate`는 금리 변동에 따른 대출 조건 및 투자 심리를 반영하여, 부동산 시장의 전반적인 가격 변동을 설명할 수 있습니다.
  
- **파생 및 상호작용 변수**:  
  - `area_x_year`, `area_x_floor` 등의 상호작용 변수는 단순 선형 관계를 넘어, 변수들 간의 복합적인 영향을 모델에 반영할 수 있도록 도와줍니다.
  
- **범주형 변수 (브랜드, 공구)**:  
  - 아파트 브랜드와 K-Means를 통해 도출한 `zone` 변수는 부동산 시장에서 지역별, 브랜드별 차이를 반영하여 세분화된 분석이 가능하게 합니다.
  
- **이벤트 및 외부 요인 변수**:  
  - `gtx_issue`와 같이 특정 인프라 이벤트를 반영한 더미 변수는, 외부 요인이 부동산 거래에 미치는 영향을 시간적으로 평가할 수 있도록 합니다.
  
- **주택 보급률**:  
  - 공급 측면을 반영하는 주택 보급률 변수는, 인구 대비 주택 공급 상태를 분석하여 시장 과열 혹은 침체 여부를 진단하는 데 기여합니다.

---

## 결론

전체 데이터 분석 흐름은 다음과 같은 논리적 단계로 진행됩니다.

1. **데이터 수집 및 전처리**:  
   - 여러 출처에서 데이터를 수집하고, 각각의 데이터셋에 대해 결측치 처리, 데이터 타입 변환, 파생 변수 생성 등 기본 전처리 과정을 수행합니다.

2. **EDA 및 개별 분석**:  
   - 수집된 데이터를 시각화 및 통계 분석(분포, 상관관계, 회귀 등)을 통해 주요 변수의 특성과 이상치를 파악합니다.

3. **Feature Engineering**:  
   - 거리 기반 변수, 상호작용 변수, 이벤트 더미 변수 등 다양한 파생 변수를 생성하여, 부동산 가격에 영향을 주는 다양한 요인을 반영합니다.

4. **데이터 통합 및 최종 모델링**:  
   - 모든 전처리된 데이터를 공통 키를 기준으로 병합하고, 통합된 데이터셋을 바탕으로 시계열 분석, 공간 분석, 회귀 분석 등을 수행합니다.

이러한 체계적인 흐름과 변수 관리는 부동산 가격 결정 요인을 다각도로 분석할 수 있게 해주며, 모델의 설명력과 예측력을 높이는 데 중요한 역할을 합니다. 각 단계에서 변수의 의미와 역할, 그리고 상호작용을 면밀하게 고려함으로써, 현실적이고 객관적인 분석 결과를 도출할 수 있는 기반을 마련하고 있습니다.